In [1]:
import osmnx as ox, networkx as nx, time, pandas as pd, numpy as np, matplotlib.cm as cm, matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon
ox.config(log_console=True, use_cache=True)
%matplotlib inline

In [2]:
network_type = 'drive'
retain_all = True

In [3]:
start_time = time.time()

In [4]:
wgs84 = {'init':'epsg:4326'}
gdf = gpd.read_file('input_data/ZillowNeighborhoods-CA/')

In [5]:
sf = gdf[gdf['CITY']=='San Francisco']
sf.head()

,CITY,COUNTY,NAME,REGIONID,STATE,geometry
789,San Francisco,San Francisco,Bayview,272885.0,CA,"POLYGON ((-122.380496615061 37.7507156475919, ..."
790,San Francisco,San Francisco,Bernal Heights,268020.0,CA,"POLYGON ((-122.403862539662 37.7494769720709, ..."
791,San Francisco,San Francisco,Castro-Upper Market,276241.0,CA,"POLYGON ((-122.426029676707 37.7697778521009, ..."
792,San Francisco,San Francisco,Chinatown,114291.0,CA,"POLYGON ((-122.41020215338 37.7974876723953, -..."
793,San Francisco,San Francisco,Crocker Amazon,273404.0,CA,"POLYGON ((-122.454085201694 37.7082065558492, ..."


In [6]:
# get a color for each node
def get_color_list(n, color_map='plasma', start=0, end=1):
    return [cm.get_cmap(color_map)(x) for x in np.linspace(start, end, n)]

def get_node_colors_by_stat(G, data, start=0, end=1):
    df = pd.DataFrame(data=pd.Series(data).sort_values(), columns=['value'])
    df['colors'] = get_color_list(len(df), start=start, end=end)
    df = df.reindex(G.nodes())
    return df['colors'].tolist()

In [7]:
df = pd.DataFrame()
all_stats = {}
for label, row in sf.iterrows():
    
    name = row['NAME']
    geometry = row['geometry']
    geometry_utm, crs_utm = ox.project_geometry(geometry, crs=wgs84)
    area = geometry_utm.area
    
    # get the driving network and project it
    G = ox.graph_from_polygon(geometry, network_type=network_type, retain_all=retain_all)
    G_proj = ox.project_graph(G)
    
    # plot and save the figure
    filename = '{}_network'.format(name)
    fig, ax = ox.plot_graph(G_proj, fig_height=6,
                            node_color='#336699', node_size=15, node_zorder=2,
                            save=True, filename=filename, show=False)
    
    stats = ox.basic_stats(G, area=area)
    extended_stats = ox.extended_stats(G, connectivity=True, ecc=True, bc=True, cc=True, anc=True)
    for key in extended_stats:
        stats[key] = extended_stats[key]
    stats['area_km'] = area / 1e6 #sq m to sq km
    #stats['node_connectivity_avg_undir'] = nx.average_node_connectivity(G.to_undirected())
    
    filename = '{}_betweenness_centrality'.format(name)
    nc = get_node_colors_by_stat(G_proj, stats['betweenness_centrality'])
    fig, ax = ox.plot_graph(G_proj, fig_height=6,
                            node_color=nc, node_size=15, node_zorder=2,
                            save=True, filename=filename, show=False)
        
    all_stats[name] = stats
    df = df.append(pd.Series(data=stats, name=name))

In [8]:
cols = [col for col in df.columns if not isinstance(df.reset_index().loc[0, col], dict)]
df_display = pd.DataFrame(df[cols])

df_display['pagerank_max_node'] = df_display['pagerank_max_node'].astype(int)
df_display['pagerank_min_node'] = df_display['pagerank_min_node'].astype(int)

def clean_display(value):
    if isinstance(value, float):
        value = round(value, 3)
    return value

df_display = df_display.applymap(clean_display)
df_display.head()

,area_km,avg_neighbor_degree_avg,avg_weighted_neighbor_degree_avg,betweenness_centrality_avg,center,circuity_avg,closeness_centrality_avg,clustering_coefficient_avg,clustering_coefficient_weighted_avg,count_intersections,...,pagerank_min,pagerank_min_node,periphery,radius,self_loop_proportion,street_density_km,street_length_avg,street_length_total,street_segments_count,streets_per_node_avg
Bayview,12.859,2.918,0.036,0.021,[65300597],1.055,0.000,0.047,0.003,651.0,...,0.000,259408082,[662020555],3194.905,0.001,10623.383,118.373,136602.103,1154.0,3.081
Bernal Heights,2.992,2.754,0.047,0.029,[65343556],1.027,0.001,0.050,0.004,377.0,...,0.000,766912762,[315706802],1954.814,0.000,18675.431,90.553,55871.431,617.0,2.984
Castro-Upper Market,2.276,2.874,0.036,0.035,[65296327],1.017,0.001,0.049,0.006,231.0,...,0.001,65335951,[581074075],1384.393,0.000,17547.687,102.928,39936.167,388.0,3.315
Chinatown,0.356,2.004,0.026,0.127,[65295293],1.001,0.002,0.044,0.009,41.0,...,0.008,-2147483648,[65333988],745.463,0.000,15731.045,88.844,5597.163,63.0,3.667
Crocker Amazon,1.196,2.728,0.041,0.047,[65343941],1.023,0.001,0.047,0.010,108.0,...,0.001,597367424,[65350943],1179.792,0.000,13424.035,97.926,16059.943,164.0,3.048


In [9]:
df_display.to_csv('data/sf-nhoods.csv', index=True, encoding='utf-8')

In [20]:
# what is the max betwenness centrality in each network?
# ie, the most important node in each neighborhood has what % of shortest paths running through it?
se = pd.Series({nhood : max(all_stats[nhood]['betweenness_centrality'].values()) for nhood in all_stats})
se = se.map(lambda x: round(x, 3)).sort_values()
pd.DataFrame(se).to_csv('data/betweenness_centrality.csv', encoding='utf-8')
se

Outer Sunset           0.096
Diamond Heights        0.113
Mission                0.146
Seacliff               0.150
Pacific Heights        0.150
Bayview                0.161
Nob Hill               0.165
Inner Richmond         0.174
Marina                 0.186
Parkside               0.193
Lakeshore              0.194
Bernal Heights         0.199
Outer Richmond         0.204
Excelsior              0.206
Downtown               0.207
Glen Park              0.209
Western Addition       0.213
Financial District     0.216
Noe Valley             0.218
West Of Twin Peaks     0.232
Castro-Upper Market    0.241
Russian Hill           0.254
Potrero Hill           0.261
Visitacion Valley      0.262
Inner Sunset           0.271
Outer Mission          0.279
Ocean View             0.285
Haight-Ashbury         0.286
Presidio Heights       0.292
Crocker Amazon         0.301
South Of Market        0.310
North Beach            0.317
Chinatown              0.362
Twin Peaks             0.371
dtype: float64

In [11]:
time.time() - start_time

3602.975380420685